## Library Imports

In [1]:
from time import time
notebook_start_time = time()

In [2]:
import os
import re
import cv2
import pickle
import random as r
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader as DL
from torch.nn.utils import weight_norm as WN
from torchvision import models, transforms

from time import time
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")

## Constants and Utilities

In [3]:
SEED = 49
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

NUM_FEATURES = 1664
TRANSFORM = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406],
                                                     [0.229, 0.224, 0.225]),
                                ])

IMAGE_SIZE = 224
PATH = "../input/petfinder-pawpularity-score"
STATE_PATH = "../input/pf-d169lastblockretrain-preinit-sgd0-9"
NUM_FOLDS = len([name for name in os.listdir(STATE_PATH) if name[-2:] == "pt"])

verbose = False
DEBUG = False

sc_y = StandardScaler()

In [4]:
def breaker(num=50, char="*") -> None:
    print("\n" + num*char + "\n")


def get_filenames_and_targets(path: str) -> tuple:
    df = pd.read_csv(os.path.join(path, "train.csv"), engine="python")
    filenames = df.iloc[:, 0].copy().values
    targets  = df.iloc[:, -1].copy().values
    return filenames, targets


def get_filenames(path: str) -> np.ndarray:
    df = pd.read_csv(os.path.join(path, "test.csv"), engine="python")
    filenames  = df["Id"].copy().values
    return filenames


def get_image(path: str, name: str, size: int) -> np.ndarray:
    image = cv2.imread(os.path.join(path, name + ".jpg"), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2RGB)
    image = cv2.resize(src=image, dsize=(size, size), interpolation=cv2.INTER_AREA)
    return image


def make_submission(path: str, y_pred: np.ndarray) -> None:
    submission = pd.read_csv(os.path.join(path, "sample_submission.csv"), engine="python")
    submission["Pawpularity"] = y_pred
    submission.to_csv("./submission.csv", index=False)

## Dataset Template

In [5]:
class DS(Dataset):
    def __init__(self, base_path=None, filenames=None, image_size=None, transform=None):
        self.base_path = base_path
        self.filenames = filenames
        self.image_size = image_size
        self.transform = transform
    
    def __len__(self):
        return self.filenames.shape[0]
    
    def __getitem__(self, idx):
        image = get_image(self.base_path, self.filenames[idx], self.image_size)
        return self.transform(image)

## Build Model

In [6]:
def build_model(IL: int, seed: int):
    class Model(nn.Module):
        def __init__(self, IL=None):
            super(Model, self).__init__()

            self.features = models.densenet169(pretrained=False, progress=False)
            self.features = nn.Sequential(*[*self.features.children()][:-1])
            self.freeze()
            self.features.add_module("Adaptive Average Pool", nn.AdaptiveAvgPool2d(output_size=(1, 1)))
            self.features.add_module("Flatten", nn.Flatten())

            self.predictor = nn.Sequential()
            self.predictor.add_module("BN", nn.BatchNorm1d(num_features=IL, eps=1e-5))
            self.predictor.add_module("FC", WN(nn.Linear(in_features=IL, out_features=1)))
        
        def freeze(self):
            for params in self.parameters():
                params.requires_grad = False
            
            for names, params in self.named_parameters():
                if re.match(r"features.0.denseblock4", names, re.IGNORECASE):
                    params.requires_grad = True
                if re.match(r"features.0.norm5", names, re.IGNORECASE):
                    params.requires_grad = True
        
        def get_optimizer(self, lr=1e-3, wd=0.0):
            params = [p for p in self.parameters() if p.requires_grad]
            return optim.SGD(params, lr=lr, momentum=0.9, weight_decay=wd)
        
        def forward(self, x1, x2=None):
            if x2 is not None:
                x1 = self.features(x1)
                x2 = self.features(x2)
                return self.predictor(x1), self.predictor(x2)
            else:
                x1 = self.features(x1)
                return self.predictor(x1)
        
    if verbose:
        breaker()
        print("Building Model ...")

    torch.manual_seed(seed)
    model = Model(IL=IL)
    return model

## Predict

In [7]:
def predict_batch(model=None, dataloader=None, mode="test", path=None):
    model.load_state_dict(torch.load(path, map_location=DEVICE)["model_state_dict"])
    model.to(DEVICE)
    model.eval()

    y_pred = torch.zeros(1, 1).to(DEVICE)
    if re.match(r"valid", mode, re.IGNORECASE):
        for X, _ in dataloader:
            X = X.to(DEVICE)
            with torch.no_grad():
                output = model(X)
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    elif re.match(r"test", mode, re.IGNORECASE):
        for X in dataloader:
            X = X.to(DEVICE)
            with torch.no_grad():
                output = model(X)
            y_pred = torch.cat((y_pred, output.view(-1, 1)), dim=0)
    
    return y_pred[1:].detach().cpu().numpy()

## Submit

In [8]:
def submit():
    ts_filenames = get_filenames(PATH)
    filenames, targets = get_filenames_and_targets(PATH)
    
    Model = build_model(IL=NUM_FEATURES, seed=SEED)
    
    breaker()
    print("Making Predictions on Test Features ...")
    breaker()
    
    fold = 1
    final_y_pred = np.zeros((len(ts_filenames), 1))
    for tr_idx, va_idx in KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=SEED).split(filenames):
        print("Processing Fold {} ...".format(fold))
        
        tr_targets = targets[tr_idx]
        tr_targets = tr_targets.reshape(-1, 1)
        tr_targets = sc_y.fit_transform(tr_targets)
        
        ts_data_setup = DS(base_path=os.path.join(PATH, "test"), 
                           filenames=ts_filenames, 
                           image_size=IMAGE_SIZE, 
                           transform=TRANSFORM)
        ts_data = DL(ts_data_setup, batch_size=64, shuffle=False)
        
        y_pred = predict_batch(model=Model, dataloader=ts_data, mode="test",
                               path=os.path.join(STATE_PATH, "Fold_{}_state.pt".format(fold)))
        y_pred = sc_y.inverse_transform(y_pred)
        
        final_y_pred += y_pred
        fold += 1
    
    final_y_pred = final_y_pred / NUM_FOLDS
    
    breaker()
    print("Generating Submission File ...")
    make_submission(PATH, final_y_pred)
    breaker()

In [9]:
submit()


**************************************************

Making Predictions on Test Features ...

**************************************************

Processing Fold 1 ...
Processing Fold 2 ...
Processing Fold 3 ...
Processing Fold 4 ...
Processing Fold 5 ...
Processing Fold 6 ...
Processing Fold 7 ...
Processing Fold 8 ...
Processing Fold 9 ...
Processing Fold 10 ...

**************************************************

Generating Submission File ...

**************************************************



## End

In [10]:
breaker()
print("Notebook Rumtime : {:.2f} minutes".format((time() - notebook_start_time)/60))
breaker()


**************************************************

Notebook Rumtime : 0.51 minutes

**************************************************

